In [1]:
from sklearn.neighbors import NearestNeighbors
import transforms3d as t3d
from tqdm import tqdm

def get_pose(init_R, init_P):
    init_T= np.zeros([4,4])
    init_T[:3,:3]= init_R
    init_T[:, -1]= np.hstack((init_P, np.array([1])))
    return init_T

def get_R_and_P(T):
    return T[:3,:3], T[:-1,-1]

def get_pc_centroid(pc):
    return pc.mean(axis=0)
    
def get_corres_points(target_pc_moved, source_pc):
    kn_obj=NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(target_pc_moved)
    dist, idx= kn_obj.kneighbors(source_pc)
    return np.sum(dist.flatten()), idx.flatten()

def rotate_pc(R, pc):
    return (R@ pc.T).T

def ICP(source_pc, target_pc,number_of_yaw_splits=4, viz= False):
    yaw_split= (2*np.pi)/number_of_yaw_splits
    max_iters= 100
    filter_dist_threshold= 0.5

    sampled_target_pc= target_pc[::5]
    sampled_source_pc= source_pc[::5]
    euclidean_dist= []
    best_pose=[]

    for ii in range(number_of_yaw_splits):
        yaw_val= yaw_split*ii
        print('\n\n###########################\n\n')
        print("YAW_VALUE :: ", yaw_val)
        init_R= t3d.euler.euler2mat(0,0, yaw_val)
        init_p=  get_pc_centroid(source_pc)- get_pc_centroid(rotate_pc(init_R,target_pc) )
        # print("Initial R: \n",init_R)
        # print("Initial P: ",init_p)
        init_T= get_pose(init_R, init_p)
        print("Initial Pose:\n ",init_T)
        

        moved_target_pc= rotate_pc(init_R, sampled_target_pc)+init_p
        prev_euclidean_dist= 0
        accumulated_pose = init_T
        dist, corres_idxs = get_corres_points(moved_target_pc,sampled_source_pc)
        # print(dist)
        dist_between_corresponding_pc_pts= np.linalg.norm(sampled_source_pc-moved_target_pc[corres_idxs],axis=1)
        
        filtered_idx = np.where(dist_between_corresponding_pc_pts < filter_dist_threshold)
        
        sampled_source_pc=sampled_source_pc[filtered_idx]
        moved_target_pc=moved_target_pc[corres_idxs][filtered_idx]
        for itr in tqdm(range(max_iters)):
            dist, corres_idxs = get_corres_points(moved_target_pc,sampled_source_pc)

            #Kabasch stuff
            src_centroid = get_pc_centroid(sampled_source_pc)
            tar_centroid = get_pc_centroid(moved_target_pc)
        
            centered_sampled_source_pc= sampled_source_pc - src_centroid
            centered_moved_target_pc= moved_target_pc- tar_centroid
        
            Q= np.matmul(centered_sampled_source_pc.transpose(), centered_moved_target_pc[corres_idxs])
            U, S, Vt = np.linalg.svd(Q, full_matrices=True)
            R= U@Vt
            p= src_centroid - rotate_pc(R, tar_centroid)

            pred_pose= get_pose(R,p)
            accumulated_pose= pred_pose@accumulated_pose

            moved_target_pc = rotate_pc( R, moved_target_pc) + p
            if np.abs(dist-prev_euclidean_dist) < 1e-10:
                best_dist= dist
                break
            prev_euclidean_dist = dist
        print(accumulated_pose, best_dist)
        euclidean_dist.append(best_dist)
        best_pose.append(accumulated_pose)


    best_pose_pred= best_pose[np.argmin(euclidean_dist)]
    if viz:
        visualize_icp_result(target_pc, source_pc, best_pose_pred)
    return best_pose_pred

In [ ]:
import numpy as np
from utils import read_canonical_model, load_pc, visualize_icp_result

obj_name = 'liq_container' # drill or liq_container
num_pc = 4 # number of point clouds

source_pc = read_canonical_model(obj_name)

for i in range(num_pc):
    target_pc = load_pc(obj_name, i)

    ICP(source_pc, target_pc, viz=True)

In [ ]:
import transforms3d as t3d
from tqdm import tqdm
number_of_yaw_splits= 4
yaw_split= (2*np.pi)/number_of_yaw_splits
max_iters= 100
filter_dist_threshold= 0.5

sampled_target_pc= target_pc[::5]
sampled_source_pc= source_pc[::5]
euclidean_dist= []
best_pose=[]

for ii in range(number_of_yaw_splits):
    yaw_val= yaw_split*ii
    print('\n\n###########################\n\n')
    print("YAW_VALUE :: ", yaw_val)
    init_R= t3d.euler.euler2mat(0,0, yaw_val)
    init_p=  get_pc_centroid(source_pc)- get_pc_centroid(rotate_pc(init_R,target_pc) )
    # print("Initial R: \n",init_R)
    # print("Initial P: ",init_p)
    init_T= get_pose(init_R, init_p)
    # print("Initial Pose:\n ",init_T)
    

    moved_target_pc= rotate_pc(init_R, sampled_target_pc)+init_p
    prev_euclidean_dist= 0
    accumulated_pose = init_T
    dist, corres_idxs = get_corres_points(moved_target_pc,sampled_source_pc)
    # print(dist)
    dist_between_corresponding_pc_pts= np.linalg.norm(sampled_source_pc-moved_target_pc[corres_idxs],axis=1)
    
    filtered_idx = np.where(dist_between_corresponding_pc_pts < filter_dist_threshold)
    
    sampled_source_pc=sampled_source_pc[filtered_idx]
    moved_target_pc=moved_target_pc[corres_idxs][filtered_idx]
    for itr in tqdm(range(max_iters)):
        dist, corres_idxs = get_corres_points(moved_target_pc,sampled_source_pc)

        #Kabasch stuff
        src_centroid = get_pc_centroid(sampled_source_pc)
        tar_centroid = get_pc_centroid(moved_target_pc)
    
        centered_sampled_source_pc= sampled_source_pc - src_centroid
        centered_moved_target_pc= moved_target_pc- tar_centroid
    
        Q= np.matmul(centered_sampled_source_pc.transpose(), centered_moved_target_pc[corres_idxs])
        U, S, Vt = np.linalg.svd(Q, full_matrices=True)
        R= U@Vt
        p= src_centroid - rotate_pc(R, tar_centroid)

        pred_pose= get_pose(R,p)
        accumulated_pose= pred_pose@accumulated_pose

        moved_target_pc = rotate_pc( R, moved_target_pc) + p
        if np.abs(dist-prev_euclidean_dist) < 1e-10:
            best_dist= dist
            break
        prev_euclidean_dist = dist
    # print(accumulated_pose, best_dist)
    euclidean_dist.append(best_dist)
    best_pose.append(accumulated_pose)


best_pose_pred= best_pose[np.argmin(euclidean_dist)]
visualize_icp_result(target_pc, source_pc, best_pose_pred)




###########################


YAW_VALUE ::  0.0
Initial Pose:
  [[ 1.          0.          0.         -0.58054619]
 [ 0.          1.          0.         -0.0134431 ]
 [-0.          0.          1.         -0.0444727 ]
 [ 0.          0.          0.          1.        ]]


  0%|          | 0/100 [00:00<?, ?it/s]

 13%|█▎        | 13/100 [00:00<00:04, 21.13it/s]


[[ 0.57070198  0.75051682  0.33320225 -0.35853335]
 [-0.63551817  0.66065078 -0.39957127  0.38557201]
 [-0.52001529  0.01628003  0.85400179  0.26297578]
 [ 0.          0.          0.          1.        ]] 60.39366276144702


###########################


YAW_VALUE ::  1.5707963267948966
Initial Pose:
  [[ 6.12323400e-17 -1.00000000e+00  0.00000000e+00  1.34431000e-02]
 [ 1.00000000e+00  6.12323400e-17  0.00000000e+00 -5.80546193e-01]
 [-0.00000000e+00  0.00000000e+00  1.00000000e+00 -4.44727001e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


 14%|█▍        | 14/100 [00:00<00:02, 29.60it/s]


[[-0.48012215 -0.86082058 -0.1687325   0.29986566]
 [ 0.69314197 -0.49018531  0.52846245 -0.42626506]
 [-0.53762155  0.13677095  0.8320197   0.27271061]
 [ 0.          0.          0.          1.        ]] 57.78846438259848


###########################


YAW_VALUE ::  3.141592653589793
Initial Pose:
  [[-1.00000000e+00 -1.22464680e-16  0.00000000e+00  5.80546193e-01]
 [ 1.22464680e-16 -1.00000000e+00  0.00000000e+00  1.34431000e-02]
 [-0.00000000e+00  0.00000000e+00  1.00000000e+00 -4.44727001e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


 13%|█▎        | 13/100 [00:00<00:02, 30.96it/s]


[[-0.57857396 -0.73173484 -0.36030029  0.3646459 ]
 [ 0.60505932 -0.68128476  0.4120125  -0.36885149]
 [-0.54695099  0.02037666  0.8369166   0.28167931]
 [ 0.          0.          0.          1.        ]] 59.760128953475856


###########################


YAW_VALUE ::  4.71238898038469
Initial Pose:
  [[-1.83697020e-16  1.00000000e+00 -0.00000000e+00 -1.34431000e-02]
 [-1.00000000e+00 -1.83697020e-16  0.00000000e+00  5.80546193e-01]
 [-0.00000000e+00  0.00000000e+00  1.00000000e+00 -4.44727001e-02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


 13%|█▎        | 13/100 [00:00<00:01, 60.54it/s]


[[ 0.47909798  0.85201476  0.21103547 -0.30114963]
 [-0.69804816  0.5155935  -0.49688239  0.4279043 ]
 [-0.53215965  0.09074243  0.84176714  0.27205844]
 [ 0.          0.          0.          1.        ]] 52.71105717365515
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
